In [24]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

import warnings
warnings.filterwarnings("ignore")

# Functions

**Function** \
get_factor: for each date, selected the best equities by the factor chosen \
**Attributes** \
df: original dataset \
factor_name: name of factor \
number_of_equities: number of equities to select

In [25]:
def get_factor(df, factor_name, number_of_equities):   
    temp_dict = {}

    # Get all dates in order
    dates = list(set(df['date']))

    # Get the top 10 equities for each date
    for date in dates:
        temp_dict[date] =  df.loc[df['date'] == date].sort_values(by=['date',factor_name], ascending=False)['equity'].head(number_of_equities).values

    # Create a dataframe with the top 10 equities for each date
    df_factor_chosen = pd.DataFrame.from_dict(temp_dict, orient='index').sort_index()

    return df_factor_chosen

df_returns: dataframe with date and price for each equity

In [26]:
def df_returns(df):
    # Select only equities, date and price
    df_returns = df[['equity','date','PX_LAST']]
    # Sort by equity and date
    df_returns.sort_values(by=['equity','date'], inplace=True)
    # Put the column price as a value of the pivot table
    df_returns = df_returns.pivot(index='date', columns='equity', values='PX_LAST')
    return df_returns

**Function** \
returns_of_factor: add a column into the df with the returns of each row (date) \

**Attributes** \
df_factor_chosen: df with the best n equities for each date by a factor choosen before \
df_log_returns: 

In [27]:
def returns_without_fees(df_factor_chosen, df_log_returns):
    sum_row_returns = 0
    list_sum_row_returns = []
    for index, row in df_factor_chosen.iterrows():
        for equity in row:
            sum_row_returns += np.nansum(df_log_returns.loc[index][equity])
        list_sum_row_returns.append(round(sum_row_returns,2))
        sum_row_returns = 0

    df_factor_chosen['returns'] = list_sum_row_returns
    return df_factor_chosen

In [28]:
def returns_with_fees(df_factor_chosen, df_log_returns, commission_fee):
    sum_row_returns = 0
    list_sum_row_returns = []
    for index, row in df_factor_chosen.iterrows():
        for equity in row:           
            rtn_before_commission = np.nansum(df_log_returns.loc[index][equity])
            #print(rtn_before_commission)
            rtn_after_commission = rtn_before_commission - commission_fee * (1 + rtn_before_commission)
            #print(rtn_after_commission)
            sum_row_returns += rtn_after_commission

        list_sum_row_returns.append(round(sum_row_returns,2))
        sum_row_returns = 0

    df_factor_chosen['returns'] = list_sum_row_returns
    return df_factor_chosen

**Function** \
get_information_ratio: returns the information ratio of the chosen factor \
**Attributes** \
df_factor_chosen_returns: df given by the function returns_of_factor \
benchmark_returns: 

In [29]:
def get_information_ratio(df_factor_chosen_returns, benchmark_returns):
    df_factor_chosen_returns['alpha'] = df_factor_chosen_returns['returns'] - benchmark_returns
    information_ratio = df_factor_chosen_returns['alpha'].mean() / df_factor_chosen_returns['alpha'].std()
    
    return information_ratio

**Function** \
get_df_information_ratio

In [30]:
def get_df_information_ratio(df, factors_name, number_of_equities, commission_fee, df_log_returns, benchmark_returns):
    temp_dict = {}

    for factor_name in factors_name:
        df_factor = get_factor(df, factor_name, number_of_equities)
        df_factor_returns = returns_with_fees(df_factor, df_log_returns, commission_fee).iloc[1:,:]
        information_ratio = get_information_ratio(df_factor_returns, benchmark_returns)

        temp_dict[factor_name] = [information_ratio]

    df_information_ratio = pd.DataFrame.from_dict(temp_dict, orient='index', columns=['information_ratio']).sort_values(by='information_ratio', ascending=False)
    
    return df_information_ratio

# MAIN PREVIEW

**Run all functions above**

In [31]:
# Read the data
df = pd.read_csv('data/data.csv').set_index(['equity', 'date'])
df = df.drop(df.groupby(level=0).filter(lambda equity: (equity.isna().sum(axis=1) == len(equity.columns)).any()).index)
df = df.drop(df.groupby(level=0).filter(lambda equity: equity['PX_LAST'].eq(0).any()).index).reset_index()

In [32]:
# Get the returns
df_returns = df_returns(df)

# Get the log returns
df_log_returns = df_returns[df_returns.columns].apply(lambda x: np.log(x / x.shift(1)))

# Get the benchmark of returns
benchmark_returns = df_log_returns.dropna(how='all').mean(skipna=True, axis='columns')

,information_ratio
RSI_14D,1.963077
PX_TO_BOOK_RATIO,0.325439
VOLATILITY_30D,0.223620
OPERATING_ROIC,0.205135
EBITDA_MARGIN,0.136751
PE_RATIO,0.123410
CUR_MKT_CAP,0.114912
NORMALIZED_ACCRUALS_CF_METHOD,0.069455


In [34]:
# Set variables
factors_name = ['PE_RATIO', 'EBITDA_MARGIN', 'PX_TO_BOOK_RATIO', 'NORMALIZED_ACCRUALS_CF_METHOD', 'RSI_14D', 'VOLATILITY_30D', 'CUR_MKT_CAP', 'OPERATING_ROIC']
number_of_equities = 10
commission_fee = 0.002

In [35]:
df_information_ratio = get_df_information_ratio(df, factors_name, number_of_equities, commission_fee, df_log_returns, benchmark_returns)

# estrazione dei 4 factor con information ratio più alto
factors_to_use = df_information_ratio.head(4).index.values

# aggiungo la colonna zscore aggregata per i fattori scelti
df['zscore'] = df.set_index(['equity', 'date'])[factors_to_use].groupby(level=0).apply(lambda x: zscore(x.loc[:, factors_to_use]).mean(axis=1)).values

In [38]:
df_information_ratio

,information_ratio
RSI_14D,1.895153
PX_TO_BOOK_RATIO,0.371597
VOLATILITY_30D,0.231156
OPERATING_ROIC,0.149181
PE_RATIO,0.128226
EBITDA_MARGIN,0.105261
NORMALIZED_ACCRUALS_CF_METHOD,0.083320
CUR_MKT_CAP,0.080905


In [36]:
df_zscore_information_ratio = get_df_information_ratio(df, ['zscore'], number_of_equities, commission_fee, df_log_returns, benchmark_returns)
df_zscore_information_ratio

,information_ratio
zscore,0.872233


In [44]:
filter_n_equities = [300, 150, 100]


df_filtered = df.copy()
for n_equities, factor in zip(filter_n_equities, factors_to_use[:-1]):
    eq_filter = get_factor(df_filtered, factor, n_equities)
    df_filtered = pd.concat([df_filtered.loc[(df_filtered['date'] == date) & (df_filtered['equity'].isin(equities.values))
                             ] for date, equities in eq_filter.iterrows()])


filtered_df_factor = get_factor(df, factors_to_use[-1], number_of_equities)
filtered_returns = returns_with_fees(filtered_df_factor, df_log_returns, commission_fee).iloc[1:,:]
filtered_info_ratio = get_information_ratio(filtered_returns, benchmark_returns)
filtered_info_ratio

0.1491814050625087

In [46]:
filtered_info_ratio

,0,1,2,3,4,5,6,7,8,9,returns
2003-01-31,EC FP,PUM GR,SAP GR,MDN GR,SPA BB,SAN FP,NOK1V FH,ITX SM,SNG GR,EXACT NA,-0.02
2003-02-28,EC FP,PUM GR,SAP GR,MDN GR,SPA BB,SAN FP,NOK1V FH,ITX SM,SNG GR,EXACT NA,-0.85
2003-03-31,EC FP,SAP GR,PUM GR,RAA GR,SPA BB,SAN FP,MDN GR,NOK1V FH,VAC1V FH,ITX SM,-0.02
2003-04-30,EC FP,SAP GR,PUM GR,RAA GR,SPA BB,SAN FP,MDN GR,NOK1V FH,VAC1V FH,ITX SM,1.29
2003-05-30,ZOT SM,EC FP,SAP GR,PUM GR,RAA GR,SPA BB,SAN FP,MDN GR,NOK1V FH,VAC1V FH,0.57
...,...,...,...,...,...,...,...,...,...,...,...
2011-11-30,EUR FP,ZOT SM,LCA1 GR,BBED NA,AAD GR,EVS BB,MOBB BB,RAA GR,ASML NA,BOS GR,-0.12
2011-12-30,EUR FP,ZOT SM,EVS BB,LCA1 GR,BBED NA,AAD GR,CAI IM,NSU GR,MOBB BB,FSC1V FH,-0.16
2012-01-31,EUR FP,ZOT SM,EVS BB,LCA1 GR,BBED NA,AAD GR,CAI IM,NSU GR,MOBB BB,FSC1V FH,0.54
2012-02-29,EUR FP,ZOT SM,EVS BB,LCA1 GR,BBED NA,AAD GR,CAI IM,NSU GR,MOBB BB,FSC1V FH,0.30
